# VSS Event Reviewer – Local Setup and Manual Alert Demo

This notebook:
- Configures Docker to use the disk with the most free space (optional but recommended)
- Clones the NVIDIA `video-search-and-summarization` repository into `~/video-search-and-summarization`
- Starts the **Event Reviewer (Event Verification)** stack via Docker Compose
- Prepares the alert media directory (`/tmp/alert-media-dir/media`)
- Demonstrates how to upload a local video to VST and send an alert via Alert Bridge
- Lets you view and chat with the alert in the Alert Inspector UI (`http://localhost:7860`)

> ⚠️ **Warning**: The Docker setup cell will delete `/var/lib/docker` (old images/containers) to move Docker to a larger disk. Run that cell only on machines where that is acceptable (ephemeral lab machines, dev boxes, etc.).

In [ ]:
import subprocess
import os
import json

###############################################################
# CELL 1 (OPTIONAL): Configure Docker to use the disk with the
# most free space as its data-root.
#
# What this does:
#   1. Parses `df -h` to find all mounted filesystems
#   2. Picks the mount with the largest free space (in GB)
#   3. Writes `/etc/docker/daemon.json` with:
#        { "data-root": "<best_mount>/docker" }
#   4. Stops Docker, deletes old `/var/lib/docker` (DESTRUCTIVE),
#      creates `<best_mount>/docker`, restarts Docker
#   5. Prints `Docker Root Dir` to confirm
#
# ⚠️ This will remove all existing Docker images/containers/volumes
#    under `/var/lib/docker`. On ephemeral dev machines, this is fine.
###############################################################

def get_mounts_with_free_space():
    """Return list of (mountpoint, free_gb) from `df -h`."""
    out = subprocess.check_output(
        "df -h --output=target,avail | tail -n +2", shell=True
    ).decode().strip().splitlines()
    mounts = []
    for line in out:
        line = line.strip()
        if not line:
            continue
        try:
            mount, avail = line.split()
        except ValueError:
            continue

        # Skip virtual/ephemeral mounts
        if any(mount.startswith(p) for p in ["/run", "/sys", "/proc", "/dev", "/snap"]):
            continue

        # Convert available string (e.g., '777G' or '512M') to GB
        if avail.endswith("G"):
            free_gb = float(avail[:-1])
        elif avail.endswith("M"):
            free_gb = float(avail[:-1]) / 1024.0
        else:
            # Skip other units for simplicity
            continue

        mounts.append((mount, free_gb))

    if not mounts:
        raise RuntimeError("No suitable mounts found from df output.")
    return mounts

mounts = get_mounts_with_free_space()
best_mount, best_free = sorted(mounts, key=lambda x: x[1], reverse=True)[0]

print(f"📦 Disk with largest free space: {best_mount} ({best_free:.2f} GB free)")

docker_root = os.path.join(best_mount, "docker")
print("➡️ Docker will use data-root:", docker_root)

# Prepare daemon.json
daemon_cfg = {"data-root": docker_root}
tmp_daemon = "/tmp/daemon.json.tmp"

with open(tmp_daemon, "w") as f:
    json.dump(daemon_cfg, f, indent=2)

print("📝 Writing /etc/docker/daemon.json with:", daemon_cfg)
subprocess.run(f"sudo mv {tmp_daemon} /etc/docker/daemon.json", shell=True, check=False)

print("⏹ Stopping Docker services...")
subprocess.run("sudo systemctl stop docker", shell=True)
subprocess.run("sudo systemctl stop docker.socket", shell=True)

print("🧹 Removing old /var/lib/docker (this deletes old images/containers)...")
subprocess.run("sudo rm -rf /var/lib/docker", shell=True)

print("📁 Creating new docker root:", docker_root)
subprocess.run(f"sudo mkdir -p {docker_root}", shell=True)
subprocess.run(f"sudo chmod 700 {docker_root}", shell=True)

print("▶️ Starting Docker...")
subprocess.run("sudo systemctl start docker", shell=True)

try:
    info = subprocess.check_output(
        "docker info | grep 'Docker Root Dir'", shell=True
    ).decode().strip()
    print("\n✅ Docker is now using:\n", info)
except subprocess.CalledProcessError as e:
    print("\n❌ Could not query Docker info. Is Docker running?")
    print(e)


In [ ]:
import subprocess
from pathlib import Path

########################################################
# CELL 2: Clone NVIDIA repo into ~/video-search-and-summarization
########################################################

REPO_URL = "https://github.com/NVIDIA-AI-Blueprints/video-search-and-summarization.git"
REPO_PATH = Path.home() / "video-search-and-summarization"

if REPO_PATH.exists():
    print(f"📂 Repository already exists at: {REPO_PATH}")
else:
    print(f"⬇️ Cloning repository into {REPO_PATH} ...")
    result = subprocess.run(f"git clone {REPO_URL} {REPO_PATH}", shell=True)
    if result.returncode == 0:
        print("✅ Clone complete.")
    else:
        raise RuntimeError("❌ Git clone failed.")

print("\nRepository ready:", REPO_PATH)


In [ ]:
import os, subprocess

########################################################
# CELL 3: Create ALERT_REVIEW_MEDIA_BASE_DIR
#   This must match the env var used by docker compose
#   for Event Reviewer.
########################################################

ALERT_REVIEW_MEDIA_BASE_DIR = "/tmp/alert-media-dir"
MEDIA_SUBDIR = "media"

full_media_dir = os.path.join(ALERT_REVIEW_MEDIA_BASE_DIR, MEDIA_SUBDIR)

print(f"📁 Creating alert media directory at: {full_media_dir}")
subprocess.run(f"sudo mkdir -p {full_media_dir}", shell=True)
subprocess.run(f"sudo chmod -R 777 {ALERT_REVIEW_MEDIA_BASE_DIR}", shell=True)

# Simple write test
test_file = os.path.join(full_media_dir, "touch_test_from_jupyter.txt")
try:
    with open(test_file, "w") as f:
        f.write("ok\n")
    print("✅ Write test passed:", test_file)
except Exception as e:
    print("❌ Write test FAILED:", e)

print("Alert media base dir:", ALERT_REVIEW_MEDIA_BASE_DIR)
print("Media subdir:", full_media_dir)


In [ ]:
import subprocess
from pathlib import Path

########################################################
# CELL 4: Ensure docker network vss-shared-network exists
########################################################

print("🔧 Ensuring vss-shared-network exists...")
cmd = """
docker network ls --format '{{.Name}}' | grep -w vss-shared-network || docker network create vss-shared-network
"""
subprocess.run(cmd, shell=True)

print("✅ Docker network ready.")


In [ ]:
import os, subprocess
from pathlib import Path

########################################################
# CELL 5: Start Event Reviewer (Event Verification stack)
#   Equivalent to docs:
#     cd deploy/docker/event_reviewer
#     ALERT_REVIEW_MEDIA_BASE_DIR=/tmp/alert-media-dir docker compose up -d
########################################################

REPO_PATH = Path.home() / "video-search-and-summarization"
EVENT_REVIEWER_DIR = REPO_PATH / "deploy" / "docker" / "event_reviewer"

if not EVENT_REVIEWER_DIR.exists():
    raise RuntimeError(f"❌ Cannot find event_reviewer folder at: {EVENT_REVIEWER_DIR}")

print("📂 Using Event Reviewer directory:", EVENT_REVIEWER_DIR)

env = os.environ.copy()
env["ALERT_REVIEW_MEDIA_BASE_DIR"] = "/tmp/alert-media-dir"

print("\n🚀 Starting Event Reviewer (Event Verification Only)...")
print("   ALERT_REVIEW_MEDIA_BASE_DIR=/tmp/alert-media-dir")

result = subprocess.run(
    "docker compose up -d",
    cwd=str(EVENT_REVIEWER_DIR),
    env=env,
    shell=True,
    text=True,
)

if result.returncode == 0:
    print("\n✅ Event Reviewer is starting successfully.")
    print("🔎 Alert Inspector UI: http://localhost:7860")
else:
    print("\n❌ Failed to start Event Reviewer.")
    print("Run this in a terminal for logs:")
    print(f"cd {EVENT_REVIEWER_DIR} && docker compose logs --tail=200")


In [ ]:
import os

########################################################
# CELL 6: Common configuration for VST & Alert Bridge
########################################################

HOST = "localhost"  # where docker is running

# These must match your .env / compose
STORAGE_HTTP_PORT = 32000   # VST storage-ms port
ALERT_BRIDGE_PORT = 9080    # Alert Bridge (FastAPI) port

VST_BASE_URL = f"http://{HOST}:{STORAGE_HTTP_PORT}"
ALERT_BRIDGE_BASE_URL = f"http://{HOST}:{ALERT_BRIDGE_PORT}"

ALERT_REVIEW_MEDIA_BASE_DIR = "/tmp/alert-media-dir"
MEDIA_SUBDIR = "media"

print("VST_BASE_URL:", VST_BASE_URL)
print("ALERT_BRIDGE_BASE_URL:", ALERT_BRIDGE_BASE_URL)
print("ALERT_REVIEW_MEDIA_BASE_DIR:", ALERT_REVIEW_MEDIA_BASE_DIR)


In [ ]:
import shutil
from pathlib import Path

########################################################
# CELL 7: Copy a local video into ALERT_REVIEW_MEDIA_BASE_DIR
########################################################

# EDIT this path to point to any local video file
SOURCE_VIDEO_PATH = "/home/ubuntu/sample_videos/warehouse_safety_video_short_no_ppe.mp4"

if not os.path.exists(SOURCE_VIDEO_PATH):
    raise FileNotFoundError(f"Source video not found: {SOURCE_VIDEO_PATH}")

target_filename = os.path.basename(SOURCE_VIDEO_PATH)
relative_media_path = f"{MEDIA_SUBDIR}/{target_filename}"
dest_path = os.path.join(ALERT_REVIEW_MEDIA_BASE_DIR, relative_media_path)

print("Copying local video → alert media directory:")
print("  Source:", SOURCE_VIDEO_PATH)
print("  Target:", dest_path)

os.makedirs(os.path.dirname(dest_path), exist_ok=True)
shutil.copy2(SOURCE_VIDEO_PATH, dest_path)

print("\n✅ File copied successfully.")
print("Relative media path (for VST & Alert Bridge):", relative_media_path)


In [ ]:
import json, time, requests

########################################################
# CELL 8: Register the video with VST (storage-ms)
#   Uses form-data with metadata as JSON string,
#   matching the VST behavior that expects metadata.
########################################################

metadata = {
    "sensorId": "camera-001",
    "timestamp": int(time.time()),
    "eventInfo": "manual_upload_demo",
    "streamName": "manual_demo_stream",
}

form_data = {
    "mediaFilePath": relative_media_path,
    "metaDataFilePath": "",  # no separate CV metadata file
    "metadata": json.dumps(metadata),  # JSON string
}

print("Sending payload to VST (as form data):")
print(json.dumps({
    "mediaFilePath": form_data["mediaFilePath"],
    "metaDataFilePath": form_data["metaDataFilePath"],
    "metadata": metadata
}, indent=2))

resp = requests.post(f"{VST_BASE_URL}/api/v1/storage/file", data=form_data)
print("\nStatus:", resp.status_code)
print("Response:", resp.text)

resp.raise_for_status()
vst_response = resp.json()
vst_id = vst_response["id"]

print("\n✅ VST registration complete.")
print("VST ID:", vst_id)
print("VST filePath:", vst_response.get("filePath"))


In [ ]:
from pprint import pprint
import uuid
from datetime import datetime, timezone

########################################################
# CELL 9: Send manual alert to Alert Bridge
#   This will make the alert appear in Alert Inspector UI
#   with the selected video attached, if everything is wired.
########################################################

alert_id = str(uuid.uuid4())
timestamp_iso = datetime.now(timezone.utc).strftime("%Y-%m-%dT%H:%M:%SZ")

# Use RELATIVE path for video_path; backend joins with base dir
video_rel_path = relative_media_path

vlm_params = {
    "prompt": "Describe briefly what happens in this clip.",
    "system_prompt": "You are a video reviewer. Provide a clear, concise summary.",
    "max_tokens": 128,
    "temperature": 0.3,
    "top_p": 0.3,
    "top_k": 40,
    "seed": 42,
}

vss_params = {
    "vlm_params": vlm_params,
    "cv_metadata_overlay": False,
    "enable_reasoning": False,
    "do_verification": True,
    "debug": False,
}

alert_payload = {
    "id": alert_id,
    "version": "1.0",
    "@timestamp": timestamp_iso,
    "sensor_id": "manual_demo_sensor",
    "video_path": video_rel_path,
    "vst_id": vst_id,
    "start_time": "0.0",
    "end_time": "9999.0",
    "alert": {
        "severity": "MEDIUM",
        "status": "REVIEW_PENDING",
        "type": "manual_demo_alert",
        "description": "Manual alert triggered from Jupyter.",
    },
    "event": {
        "type": "video_analysis",
        "description": "Manual demo event",
    },
    "confidence": 1.0,
    "cv_metadata_path": "",
    "vss_params": vss_params,
    "meta_labels": [
        {"key": "prompt_index", "value": "0"},
        {"key": "prompt_text", "value": vlm_params["prompt"]},
        {"key": "enable_reasoning", "value": "False"},
    ],
}

print("Sending alert to Alert Bridge with payload:")
pprint(alert_payload)

resp = requests.post(f"{ALERT_BRIDGE_BASE_URL}/api/v1/alerts", json=alert_payload)
print("\nStatus:", resp.status_code)
print("Response:", resp.text)
resp.raise_for_status()

print("\n✅ Alert successfully submitted.")
print("Alert ID:", alert_id)
print("Open UI: http://localhost:7860")


In [ ]:
########################################################
# CELL 10: Basic health checks for Alert Bridge
########################################################
import requests
try:
    print("Alert Bridge /health:")
    print(requests.get(f"{ALERT_BRIDGE_BASE_URL}/health").json())
except Exception as e:
    print("Error querying Alert Bridge /health:", e)

try:
    print("\nAlert Bridge /api/v1/alerts/health:")
    print(requests.get(f"{ALERT_BRIDGE_BASE_URL}/api/v1/alerts/health").json())
except Exception as e:
    print("Error querying Alert Bridge alerts health:", e)
